In [4]:
import json
from collections import Counter, defaultdict
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [5]:
IN_PATH = "/Users/cj2837/Documents/Courses/Project/outputs/mapped_triples.json"

with open(IN_PATH, "r") as f:
    triples = json.load(f)

In [21]:
from collections import defaultdict

# -------------------------------------------------------
# 1. Collect all unique nodes and their CUIs
# -------------------------------------------------------

node_to_cui = {}    # node → CUI or None

for t in triples:
    # head
    node_to_cui[t["head"]] = t.get("head_cui")
    # tail
    node_to_cui[t["tail"]] = t.get("tail_cui")

# -------------------------------------------------------
# 2. Compute statistics
# -------------------------------------------------------

total_nodes = len(node_to_cui)

mapped_nodes = {n for n, cui in node_to_cui.items() if cui not in (None, "", "None")}
unmapped_nodes = {n for n, cui in node_to_cui.items() if cui in (None, "", "None")}

n_mapped = len(mapped_nodes)
n_unmapped = len(unmapped_nodes)

# -------------------------------------------------------
# 3. Print results
# -------------------------------------------------------

print("=== Node Mapping Summary ===")
print(f"Total unique nodes:       {total_nodes}")
print(f"Mapped nodes (CUI found): {n_mapped}  ({n_mapped/total_nodes*100:.2f}%)")
print(f"Unmapped nodes:           {n_unmapped}  ({n_unmapped/total_nodes*100:.2f}%)")

# Optional: view a few examples
print("\nExamples of unmapped nodes:")
for n in list(unmapped_nodes)[:15]:
    print(" -", n)


=== Node Mapping Summary ===
Total unique nodes:       999
Mapped nodes (CUI found): 526  (52.65%)
Unmapped nodes:           473  (47.35%)

Examples of unmapped nodes:
 - flushing of the face and neck
 - cumulative incidence of ischemic stroke
 - serum 25-hydroxyvitamin d <20 ng/ml
 - community prevention study
 - low risk for stroke, heart disease, blood clots, or breast cancer
 - low-dose 17-ß-estradiol
 - change from baseline in greene's scale
 - women with history of blood clots
 - bone mineral density of femoral neck
 - activation frequency in endocortical compartment
 - number of participants with absolute reduction of 5 in mean number of mild, moderate and severe vms per day
 - longer than necessary
 - cortical volumetric bmd of the hip
 - serum c-terminal propeptide of type 1 collagen
 - low-dose estradiol mht


In [22]:
mapped_list = sorted(list(mapped_nodes))


In [ ]:
import json
import time
from openai import RateLimitError

def llm_batch_match(unmapped_terms, mapped_candidates, retries=2):

    CANDIDATE_STR = "\n".join(f"- {t}" for t in mapped_candidates)
    TERMS_STR = "\n".join(f"- {t}" for t in unmapped_terms)

    prompt = f"""
You are normalizing biomedical concept names extracted from a menopause-related knowledge graph.

Below is a list of previously mapped, validated biomedical concepts (the candidate list),
and a list of unmapped concept terms that require normalization.

Your task:

- For EACH unmapped term, select the single most appropriate concept from the mapped list.
- Only select a match if the unmapped term is a clear synonym, lexical variant, abbreviation,
  or minor formatting variation of a candidate concept.
- Consider differences due to:
    - capitalization
    - pluralization
    - spelling variants
    - abbreviations or expansions
    - minor wording differences referring to the same biomedical concept

STRICT RULES (must be followed exactly):

- You MUST choose ONLY from the provided mapped list.
- You MUST NOT generate new concepts, paraphrases, or inferred terms.
- If no reasonable match exists, output "NONE" for that term.
- Do NOT perform semantic generalization or abstraction.
- Do NOT merge related but distinct biomedical concepts.
- Do NOT output explanations, comments, or additional text.
- Output MUST be a single valid JSON list and nothing else.

Mapped concepts:
{CANDIDATE_STR}

Unmapped concepts:
{TERMS_STR}

Return JSON exactly in this format:
[
  {{"term": "...", "best_match": "..." }},
  ...
]
"""

    for attempt in range(retries+1):
        try:
            resp = client.chat.completions.create(
                model=os.getenv("OPENAI_MODEL_NAME", "gpt-4o"),
                temperature=0,
                messages=[
                    {"role": "system", "content": "Return ONLY valid JSON."},
                    {"role": "user", "content": prompt}
                ]
            )

            raw = resp.choices[0].message.content.strip()

            # --- Clean output ---
            cleaned = raw.replace("```json", "").replace("```", "").strip()

            try:
                return json.loads(cleaned)
            except json.JSONDecodeError:
                print("⚠️ JSON decode failed, showing raw output:")
                print(raw)
                if attempt < retries:
                    print("⏳ Retrying...\n")
                    time.sleep(1)
                continue

        except RateLimitError:
            print("⏳ Rate limit hit — waiting 4 seconds...")
            time.sleep(4)
            continue

    print("❌ Failed to parse JSON after retries — returning empty list")
    return []


In [24]:
normalized_mapping = {}

for i in range(0, len(unmapped_list), batch_size):
    batch = unmapped_list[i : i + batch_size]
    print(f"🔍 Processing batch {i//batch_size + 1} ({len(batch)} terms)...")

    results = llm_batch_match(batch, mapped_list)

    for item in results:
        term = item["term"]
        best = item["best_match"]
        if best != "NONE":
            normalized_mapping[term] = best


🔍 Processing batch 1 (40 terms)...
🔍 Processing batch 2 (40 terms)...
🔍 Processing batch 3 (40 terms)...
🔍 Processing batch 4 (40 terms)...
🔍 Processing batch 5 (40 terms)...
🔍 Processing batch 6 (40 terms)...
🔍 Processing batch 7 (40 terms)...
🔍 Processing batch 8 (40 terms)...
🔍 Processing batch 9 (40 terms)...
🔍 Processing batch 10 (40 terms)...
🔍 Processing batch 11 (40 terms)...
🔍 Processing batch 12 (33 terms)...


In [25]:
print("\n=== LLM Normalization Results ===")
print("Total unmapped:", len(unmapped_nodes))
print("Successfully normalized:", len(normalized_mapping))

print("\nExamples of normalized mappings:")
for k, v in list(normalized_mapping.items())[:15]:
    print(f"  {k}  →  {v}")



=== LLM Normalization Results ===
Total unmapped: 473
Successfully normalized: 375

Examples of normalized mappings:
  flushing of the face and neck  →  Flushing
  cumulative incidence of ischemic stroke  →  Cerebrovascular accident
  serum 25-hydroxyvitamin d <20 ng/ml  →  Vitamin D
  low-dose 17-ß-estradiol  →  Estradiol
  women with history of blood clots  →  Blood Clot
  bone mineral density of femoral neck  →  Bone Density
  number of participants with absolute reduction of 5 in mean number of mild, moderate and severe vms per day  →  Vasomotor menopausal symptoms
  cortical volumetric bmd of the hip  →  Bone Density
  serum c-terminal propeptide of type 1 collagen  →  Serum procollagen type I N-terminal propeptide increased
  low-dose estradiol mht  →  Estradiol
  endometrial protection  →  Endometrium
  hormone levels change randomly  →  Finding of hormone level
  new vertebral fractures  →  Spinal Fractures
  change from baseline in serum procollagen type i n-terminal propepti

## Check

In [26]:
normalized_mapping

{'flushing of the face and neck': 'Flushing',
 'cumulative incidence of ischemic stroke': 'Cerebrovascular accident',
 'serum 25-hydroxyvitamin d <20 ng/ml': 'Vitamin D',
 'low-dose 17-ß-estradiol': 'Estradiol',
 'women with history of blood clots': 'Blood Clot',
 'bone mineral density of femoral neck': 'Bone Density',
 'number of participants with absolute reduction of 5 in mean number of mild, moderate and severe vms per day': 'Vasomotor menopausal symptoms',
 'cortical volumetric bmd of the hip': 'Bone Density',
 'serum c-terminal propeptide of type 1 collagen': 'Serum procollagen type I N-terminal propeptide increased',
 'low-dose estradiol mht': 'Estradiol',
 'endometrial protection': 'Endometrium',
 'hormone levels change randomly': 'Finding of hormone level',
 'new vertebral fractures': 'Spinal Fractures',
 'change from baseline in serum procollagen type i n-terminal propeptide': 'Serum procollagen type I N-terminal propeptide increased',
 'nonprescription, water-based lubricant

In [28]:
hallucinations = {}

for unmapped, predicted in normalized_mapping.items():
    if predicted not in mapped_nodes:
        hallucinations[unmapped] = predicted

print("=== Normalization Hallucination Report ===")
print(f"Total unmapped terms sent to LLM: {len(normalized_mapping)}")
print(f"Hallucinated matches:              {len(hallucinations)}")
print(f"Hallucination rate:                {len(hallucinations)/len(normalized_mapping)*100:.2f}%\n")

print("Examples of hallucinated mappings:")
for u, p in list(hallucinations.items()):
    print(f" - '{u}' → '{p}' (NOT in mapped list!)")


=== Normalization Hallucination Report ===
Total unmapped terms sent to LLM: 375
Hallucinated matches:              21
Hallucination rate:                5.60%

Examples of hallucinated mappings:
 - 'serum 25-hydroxyvitamin d <20 ng/ml' → 'Vitamin D' (NOT in mapped list!)
 - 'low-dose 17-ß-estradiol' → 'Estradiol' (NOT in mapped list!)
 - 'low-dose estradiol mht' → 'Estradiol' (NOT in mapped list!)
 - 'nonprescription, water-based lubricant' → 'Vaginal lubricant water-based' (NOT in mapped list!)
 - 'bone formation rate in endocortical compartment' → 'Bone formation rate' (NOT in mapped list!)
 - 'custom-mixed bioidentical hormones' → 'Bioidentical hormone replacement therapy' (NOT in mapped list!)
 - 'over-the-counter vaginal moisturizer' → 'Vaginal moisturizer' (NOT in mapped list!)
 - 'obese menopausal women veterans with prediabetes and early diabetes' → 'At increased risk of diabetes' (NOT in mapped list!)
 - 'loss of height due to bone loss' → 'Bone loss' (NOT in mapped list!)
 -

In [31]:
valid_matches = {
    u: p for u, p in normalized_mapping.items()
    if p in mapped_nodes
}

print("=== Valid Normalizations ===")
print(f"Valid normalized terms: {len(valid_matches)}")
print(f"Coverage of unmapped resolved: {len(valid_matches)/len(unmapped_nodes)*100:.2f}%")


=== Valid Normalizations ===
Valid normalized terms: 354
Coverage of unmapped resolved: 74.84%


In [32]:
print("=== Coverage Report ===")
print(f"Unmapped nodes total:         {len(unmapped_nodes)}")
print(f"LLM provided matches for:     {len(normalized_mapping)}")
print(f"No-good-match (model said NONE): {len(unmapped_nodes) - len(normalized_mapping)}")


=== Coverage Report ===
Unmapped nodes total:         473
LLM provided matches for:     375
No-good-match (model said NONE): 98


#### concept normalization validity accuracy

In [33]:
import random

# valid normalization pairs (already computed earlier)
valid_matches = {
    u: p for u, p in normalized_mapping.items()
    if p in mapped_nodes
}

# Convert to list for sampling
valid_items = list(valid_matches.items())

# Compute sample size (10%)
sample_size = max(1, int(len(valid_items) * 0.10))

# Random sampling without replacement
sample = random.sample(valid_items, sample_size)

print("=== Random 10% Sample for Manual Validation ===")
print(f"Sample size: {sample_size} out of {len(valid_items)} valid mappings\n")

for unmapped, mapped in sample:
    print(f"- UNMAPPED: \"{unmapped}\"")
    print(f"  → MAPPED: \"{mapped}\"\n")


=== Random 10% Sample for Manual Validation ===
Sample size: 35 out of 354 valid mappings

- UNMAPPED: "feel stressed or irritable"
  → MAPPED: "Irritability - emotion"

- UNMAPPED: "urinary deoxypyridinoline/creatinine ratio"
  → MAPPED: "Increased urinary type 1 collagen N-terminal telopeptide level"

- UNMAPPED: "average daily severity of hot flushes"
  → MAPPED: "Hot flushes"

- UNMAPPED: "oral calcitonin"
  → MAPPED: "salmon calcitonin Nasal Spray"

- UNMAPPED: "volumetric bone mineral density of hip"
  → MAPPED: "Bone Mineral Density Test"

- UNMAPPED: "number of participants with absolute reduction of 2 in mean number of mild, moderate and severe vms per day"
  → MAPPED: "Vasomotor menopausal symptoms"

- UNMAPPED: "self-reported sleep quality"
  → MAPPED: "Quality of sleep"

- UNMAPPED: "changing what you eat"
  → MAPPED: "Diet"

- UNMAPPED: "post-menopausal women"
  → MAPPED: "Postmenopause"

- UNMAPPED: "bonviva treatment"
  → MAPPED: "Bonviva"

- UNMAPPED: "trabecular bmd at

3/35 are inaccurate

In [34]:
# unmapped_nodes = set of all nodes that failed original UMLS lookup
# normalized_mapping = dict of unmapped → LLM-proposed mapped match (may be hallucinated)

valid_matches = {
    u: p for u, p in normalized_mapping.items()
    if p in mapped_nodes  # ONLY real mapped concepts
}

hallucinated_matches = {
    u: p for u, p in normalized_mapping.items()
    if p not in mapped_nodes  # LLM-invented or invalid matches
}

# Truly unmapped = unmapped nodes that are not valid matches
resolved_unmapped = set(valid_matches.keys())
truly_unmapped = unmapped_nodes - resolved_unmapped

print("=== Concept Normalization Summary ===\n")
print(f"Total unmapped nodes:          {len(unmapped_nodes)}")
print(f"LLM returned a match for:      {len(normalized_mapping)}")
print(f"✔️ Valid matches:               {len(valid_matches)}")
print(f"⚠️ Hallucinated matches:        {len(hallucinated_matches)}")
print(f"❌ Still unmapped after LLM:    {len(truly_unmapped)}")
print(f"Coverage of successful mapping: {len(valid_matches)/len(unmapped_nodes)*100:.2f}%")


=== Concept Normalization Summary ===

Total unmapped nodes:          473
LLM returned a match for:      375
✔️ Valid matches:               354
⚠️ Hallucinated matches:        21
❌ Still unmapped after LLM:    119
Coverage of successful mapping: 74.84%


In [35]:
# valid_matches = {u → p} where p ∈ mapped_nodes

# 1. Check which "u" values are truly from unmapped_nodes
valid_real = {
    u: p for u, p in valid_matches.items()
    if u in unmapped_nodes
}

# 2. Find fabricated "u" that the LLM invented
valid_fake = {
    u: p for u, p in valid_matches.items()
    if u not in unmapped_nodes
}

print("=== Validation of LLM-normalized keys ===\n")
print(f"Total valid matches:                 {len(valid_matches)}")
print(f" - Valid keys from unmapped_nodes:   {len(valid_real)}")
print(f" - ❌ Fabricated keys (LLM-made-up):   {len(valid_fake)}\n")

if valid_fake:
    print("Hallucinated (invented) unmapped terms:")
    for u, p in valid_fake.items():
        print(f"  '{u}' → matched to '{p}'")
else:
    print("✔️ No hallucinated unmapped terms — all terms are real.")


=== Validation of LLM-normalized keys ===

Total valid matches:                 354
 - Valid keys from unmapped_nodes:   354
 - ❌ Fabricated keys (LLM-made-up):   0

✔️ No hallucinated unmapped terms — all terms are real.


## Replace

In [36]:
# Build a lookup: mapped term → UMLS info
mapped_info = {
    node: {"CUI": cui, "preferred": node} 
    for node, cui in node_to_cui.items()
    if cui not in (None, "", "None")
}

# The preferred name is already the standardized 'head'/'tail' value
# If you want to use the exact UMLS preferred term, adjust above.

updated_triples = []

for t in triples:
    
    # --- HEAD ---
    head = t["head"]
    if head in valid_matches:
        # normalized head
        new_head = valid_matches[head]
        new_head_cui = mapped_info[new_head]["CUI"]
    else:
        # keep original
        new_head = head
        new_head_cui = t.get("head_cui")
    
    # --- TAIL ---
    tail = t["tail"]
    if tail in valid_matches:
        new_tail = valid_matches[tail]
        new_tail_cui = mapped_info[new_tail]["CUI"]
    else:
        new_tail = tail
        new_tail_cui = t.get("tail_cui")

    updated_triples.append({
        **t,                       # keep all original fields
        "head": new_head,
        "head_cui": new_head_cui,
        "tail": new_tail,
        "tail_cui": new_tail_cui
    })


In [37]:
print("=== Changed head nodes ===")
for t_old, t_new in zip(triples, updated_triples):
    if t_old["head"] != t_new["head"]:
        print(f"{t_old['head']}  →  {t_new['head']}")

print("\n=== Changed tail nodes ===")
for t_old, t_new in zip(triples, updated_triples):
    if t_old['tail'] != t_new['tail']:
        print(f"{t_old['tail']}  →  {t_new['tail']}")


=== Changed head nodes ===
postmenopausal women with osteopenia  →  Osteopenia
progestogenic symptoms  →  Progestins
progestogenic symptoms  →  Progestins
progestogenic symptoms  →  Progestins
progestogenic symptoms  →  Progestins
progestogenic symptoms  →  Progestins
progestogenic symptoms  →  Progestins
serum c-terminal telopeptide of type 1 collagen  →  C-telopeptide serum
korean women with postmenopausal osteoporosis  →  Osteoporosis, Postmenopausal
korean women with postmenopausal osteoporosis  →  Osteoporosis, Postmenopausal
post-menopausal women  →  Postmenopause
postmenopausal women with low bone density  →  Osteoporosis, Postmenopausal
oral calcitonin  →  salmon calcitonin Nasal Spray
change in plasma c-terminal telopeptide of collagen 1  →  C-telopeptide serum
change in bone mineral density of axial lumbar spine  →  Lumbar spine DXA scan T score (finding)
perimenopausal women  →  Perimenopause
perimenopausal women  →  Perimenopause
perimenopausal women  →  Perimenopause
chang

In [38]:
# ----------------------------------------
# 1. Count how many nodes changed
# ----------------------------------------
changed_heads = 0
changed_tails = 0

for old, new in zip(triples, updated_triples):
    if old["head"] != new["head"]:
        changed_heads += 1
    if old["tail"] != new["tail"]:
        changed_tails += 1

print("=== Node Normalization Summary ===")
print(f"Changed head nodes: {changed_heads}")
print(f"Changed tail nodes: {changed_tails}")
print(f"Total changed nodes (any side): {changed_heads + changed_tails}")


=== Node Normalization Summary ===
Changed head nodes: 177
Changed tail nodes: 343
Total changed nodes (any side): 520


In [40]:
# ----------------------------------------
# 2. Count unique normalized nodes
# ----------------------------------------
normalized_nodes = set()

for t in updated_triples:
    normalized_nodes.add(t["head"])
    normalized_nodes.add(t["tail"])

print("\n=== Unique Node Count After Normalization ===")
print(f"Total unique nodes: {len(normalized_nodes)}")

# Optional: print a few examples
print("\nExamples of normalized nodes:")
for n in list(normalized_nodes)[40:60]:
    print(" -", n)



=== Unique Node Count After Normalization ===
Total unique nodes: 645

Examples of normalized nodes:
 - nonprescription, water-based lubricant
 - Stroke (heart beat)
 - carrying a portable fan
 - Pattern of menstrual cycle
 - Cardiovascular Diseases
 - cogmax
 - moodiness
 - Antidepressive Agents
 - GLUCOSE CONTROL
 - increased arterial stiffness
 - No history of depression
 - National Heart, Lung, and Blood Institute (U.S.)
 - Feel Worn Out or Sluggish Question
 - Female Pelvic Medicine and Reconstructive Surgery Urologist
 - Identify risk factors (e.g., diabetes, depressed immune system, vascular disease, nutritional deficiencies, smoking, immobility)
 - ethanol
 - nccih-funded researchers
 - Vaginal pH
 - fezolinetant
 - Closure of defect of atrioventricular septum using dual prosthetic patches


## Keep 1 relation for duplicate triples

In [41]:
from collections import Counter, defaultdict

# -------------------------------------------
# 1. Use updated_triples as input
# -------------------------------------------
relation_counts = Counter(t["relation"] for t in updated_triples)

# Map (head, tail) → list of relations
pair_relations = defaultdict(list)
for t in updated_triples:
    pair_relations[(t["head"], t["tail"])].append(t["relation"])

# Identify head–tail pairs with multiple relations
conflicts = {k: set(v) for k, v in pair_relations.items() if len(set(v)) > 1}
print(f"Found {len(conflicts)} conflicting head–tail pairs.\n")


Found 39 conflicting head–tail pairs.



In [42]:
def pick_best_relation(rel_set):
    rel_list = list(rel_set)

    # Rule 1: drop "affects" if there are more specific relations
    if "affects" in rel_set and len(rel_set) > 1:
        rel_list = [r for r in rel_list if r != "affects"]

    # Rule 2: sort by global frequency (descending)
    rel_list.sort(key=lambda r: relation_counts[r], reverse=True)

    # Rule 3: if tie, lexicographic order acts as tie-breaker
    return rel_list[0]


In [43]:
# Build dictionary: (head, tail) → selected canonical relation
best_relation_for_pair = {
    (head, tail): pick_best_relation(relations)
    for (head, tail), relations in conflicts.items()
}


In [44]:
cleaned_triples = []

for t in updated_triples:
    key = (t["head"], t["tail"])

    if key in best_relation_for_pair:
        # keep only selected best relation
        if t["relation"] == best_relation_for_pair[key]:
            cleaned_triples.append(t)
    else:
        # no conflict → keep as is
        cleaned_triples.append(t)


In [45]:
print(f"Original triples: {len(updated_triples)}")
print(f"After resolving relation conflicts: {len(cleaned_triples)}")
print(f"Removed {len(updated_triples) - len(cleaned_triples)} redundant triples.")


Original triples: 1197
After resolving relation conflicts: 1139
Removed 58 redundant triples.


In [46]:
print("\n=== Relation Conflict Resolution Report ===\n")
for (head, tail), kept in best_relation_for_pair.items():
    rels = sorted(list(conflicts[(head, tail)]))
    print(f"- {head}  ↔  {tail}")
    print(f"  Relations: {rels}  →  kept **{kept}**\n")



=== Relation Conflict Resolution Report ===

- Bonviva  ↔  Bone Mineral Density Test
  Relations: ['affects', 'improves', 'increases']  →  kept **increases**

- AMG 162  ↔  Increased bone mineral density of lumbar spine
  Relations: ['affects', 'prevents']  →  kept **prevents**

- ibandronate  ↔  C-telopeptide serum
  Relations: ['affects', 'suppresses']  →  kept **suppresses**

- Osteoporosis, Postmenopausal  ↔  ibandronate
  Relations: ['receives', 'treated_with']  →  kept **receives**

- Osteoporosis, Postmenopausal  ↔  risedronate
  Relations: ['receives', 'treated_with']  →  kept **receives**

- denosumab  ↔  C-telopeptide serum
  Relations: ['affects', 'decreases']  →  kept **decreases**

- Estrogen measurement  ↔  Pituitary Diseases
  Relations: ['affects', 'influences']  →  kept **influences**

- odanacatib  ↔  Bone Specific Alkaline Phosphatase Measurement
  Relations: ['decreases', 'reduces']  →  kept **reduces**

- odanacatib  ↔  C-telopeptide serum
  Relations: ['decreases

In [48]:
unique_triples = []
seen = set()

for t in cleaned_triples:     # cleaned_triples = output after all processing
    key = (t["head"], t["relation"], t["tail"])
    if key not in seen:
        unique_triples.append(t)
        seen.add(key)

print(f"Before deduplication: {len(cleaned_triples)}")
print(f"After deduplication:  {len(unique_triples)}")
print(f"Removed {len(cleaned_triples) - len(unique_triples)} duplicates.")

Before deduplication: 1139
After deduplication:  1052
Removed 87 duplicates.


In [49]:
from collections import Counter

# ------------------------------------------------------------
# 1. Final number of triples
# ------------------------------------------------------------
num_triples = len(unique_triples)
print(f"📌 Final number of triples: {num_triples}")

# ------------------------------------------------------------
# 2. Unique nodes
# ------------------------------------------------------------
nodes = set()

for t in unique_triples:
    nodes.add(t["head"])
    nodes.add(t["tail"])

num_nodes = len(nodes)
print(f"📌 Number of unique nodes: {num_nodes}")

# ------------------------------------------------------------
# 3. Unique edges (head–tail pairs, regardless of relation)
# ------------------------------------------------------------
edges = set()

for t in unique_triples:
    edges.add((t["head"], t["tail"]))

num_edges = len(edges)
print(f"📌 Number of unique edges (head → tail): {num_edges}")

# ------------------------------------------------------------
# 4. Unique relations
# ------------------------------------------------------------
relations = set(t["relation"] for t in unique_triples)
num_relations = len(relations)
print(f"📌 Number of unique relations: {num_relations}")

# ------------------------------------------------------------
# 5. Optional: degree statistics
# ------------------------------------------------------------
out_degree = Counter(t["head"] for t in unique_triples)
in_degree  = Counter(t["tail"] for t in unique_triples)

print("\n📈 Top 10 nodes by OUT-degree (head → many):")
for node, d in out_degree.most_common(10):
    print(f"  {node}: {d}")

print("\n📈 Top 10 nodes by IN-degree (many → tail):")
for node, d in in_degree.most_common(10):
    print(f"  {node}: {d}")


📌 Final number of triples: 1052
📌 Number of unique nodes: 645
📌 Number of unique edges (head → tail): 1052
📌 Number of unique relations: 269

📈 Top 10 nodes by OUT-degree (head → many):
  Menopause: 74
  Hormone replacement therapy: 34
  Menopausal and other perimenopausal disorders due to naturally occurring (age-related) menopause and perimenopause: 30
  Symptoms such as flushing, sleeplessness, headache, lack of concentration, associated with premature menopause: 22
  Estrogen gels, creams, or sprays on skin: 21
  Hormone Therapy: 18
  Perimenopause: 16
  women's health initiative: 16
  teriparatide: 15
  Estrogen measurement: 14

📈 Top 10 nodes by IN-degree (many → tail):
  Hot flushes: 40
  Menopause: 26
  Symptoms such as flushing, sleeplessness, headache, lack of concentration, associated with premature menopause: 25
  Osteoporosis: 21
  Vaginal dryness: 19
  Premature Menopause: 16
  Increased bone mineral density of lumbar spine: 15
  Menopausal symptom: 15
  Mental Depression

In [50]:
import json
import os

OUT_PATH = "/Users/cj2837/Documents/Courses/Project/outputs/standardized_nodes_triples.json"

# cleaned_triples = your final list after:
# - parenthesis removal
# - UMLS mapping
# - LLM normalization
# - relation trimming
# - deduplication

with open(OUT_PATH, "w") as f:
    json.dump(unique_triples, f, indent=2, ensure_ascii=False)

print("✅ Final KG saved:", OUT_PATH)
print("📌 Total triples saved:", len(unique_triples))


✅ Final KG saved: /Users/cj2837/Documents/Courses/Project/outputs/standardized_nodes_triples.json
📌 Total triples saved: 1052
